In [ ]:
!pip install -qqq -U streamlit
!npm install -qqq -U localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 771.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.0 MB/s eta 0:00:00

added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [ ]:
!pip install Pillow

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 10.7 MB/s eta 0:00:00


In [ ]:
!pip install gtts

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile app.py

import streamlit as st
import numpy as np
import pandas as pd
from PIL import Image
import emoji
import tempfile
import base64

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from gtts import gTTS


# Load the best trained model
@st.cache_resource()
def load_trained_model():
  model = load_model("/content/drive/MyDrive/cnn-thfood/mobile178_kmodel.keras")
  return model

model = load_trained_model()

# Define class names, english translation, and spicy level information
mapped_df = pd.DataFrame()
class_names = ['BitterMelonSoup', 'BooPadPongali', 'CurriedFishCake', 'Dumpling', 'EggsStewed', 'FriedChicken', 'FriedKale', 'FriedMusselPancakes', 'GaengJued', 'GaengKeawWan', 'GaiYang', 'GoongObWoonSen', 'GoongPao', 'GrilledQquid', 'HoyKraeng', 'HoyLaiPrikPao', 'Joke', 'KaiJeowMooSaap', 'KaiThoon', 'KaoManGai', 'KaoMooDang', 'KhanomJeenNamYaKati', 'KhaoMokGai', 'KhaoMooTodGratiem', 'KhaoNiewMaMuang', 'KkaoKlukKaphi', 'KorMooYang', 'KuaKling', 'KuayJab', 'KuayTeowReua', 'LarbMoo', 'MassamanGai', 'MooSatay', 'NamTokMoo', 'PadPakBung', 'PadPakRuamMit', 'PadThai', 'PadYordMala', 'PhatKaphrao', 'PorkStickyNoodles', 'Roast_duck', 'Roast_fish', 'Somtam', 'SonInLawEggs', 'StewedPorkLeg', 'Suki', 'TomKhaGai', 'TomYumGoong', 'YamWoonSen', 'Yentafo']
th_names = ['ต้มฟัก', 'ปูผัดผงกะหรี่', 'ทอดมันปลา', 'ขนมจีบ', 'ไข่พะโล้', 'ไก่ทอด', 'ผัดคะน้า', 'หอยทอด', 'แกงจืด', 'แกงเขียวหวาน', 'ไก่ย่าง', 'กุ้งอบวุ้นเส้น', 'กุ้งเผา', 'ปลาหมึกย่าง', 'หอยแครงลวก', 'หอยลายผัดพริกเผา', 'โจ๊ก', 'ไข่เจียวหมูสับ', 'ไข่ตุ๋น', 'ข้าวมันไก่', 'ข้าวหมูแดง', 'ขนมจีนน้ำยากระทิ', 'ข้าวหมกไก่', 'ข้าวหมูทอดกระเทียม', 'ข้าวเหนียวมะม่วง', 'ข้าวคลุกกะปิ', 'คอหมูย่าง', 'คั่วกลิ้ง', 'ก๋วยจั๊บ', 'ก๋วยเตี๋ยวเรือ', 'ลาบหมู', 'มัสมั่นไก่', 'หมูสะเต๊ะ', 'น้ำตกหมู', 'ผัดผักบุ้ง', 'ผัดผักรวมมิตร', 'ผัดไท', 'ผัดยอดมะระ', 'ผัดกะเพรา', 'ราดหน้าหมู', 'เป็ดย่าง', 'ปลาเผา', 'ส้มตำ', 'ไข่ลูกเขย', 'ขาหมู', 'สุกี้', 'ต้มข่าไก่', 'ต้มยำกุ้ง', 'ยำวุ้นเส้น', 'เย็นตาโฟ']
th_pronounce = ['Dtom-Fuk', 'Bpoo-Phad-Phong-Gra-Rhee', 'Tord-Man-Bplaa', 'Kha-Nhom-Geeb', 'Khai-Pha-Lho', 'Gai-Tord', 'Phad-Kha-Naa', 'Hoi-Tord', 'Gaeng-Jued', 'Gaeng-Keaw-Waan', 'Gai-Yaang', 'Goong-Ob-Woon-Sen', 'Goong-Phao', 'Bplaa-Muek-Yaang', 'Hoy-Kraeng-Luak', 'Hoy-Lai-Phad-Phrik-Phao', 'Joke', 'Khai-Jeow-Mhoo-Sap', 'Khai-Dtoon', 'Khao-Man-Gai', 'Khao-Mhoo-Dang', 'Kha-Nhom-Jeen-Nam-Yaa-Kra-Thi', 'Khao-Mhok-Gai', 'Khao-Moo-Tord-Gra-Tiam', 'Khao-Nhiew-Ma-Muang', 'Khao-Kluk-Ka-Bpi', 'Kor-Mhoo-Yaang', 'Kua-Gling', 'Khuay-Jab', 'Khuay-Dtiaw-Reua', 'Laab-Mhoo', 'Mas-Sa-Man-Gai', 'Mhoo-Sa-Dtey', 'Nam-Dtok-Mhoo', 'Phad-Phak-Bung', 'Phad-Phak-Ruam-Mit', 'Phad-Thai', 'Phad-Yord-Ma-Ra', 'Phad-Kra-Phrao', 'Raad-Nhaa-Mhoo', 'Bped-Yaang', 'Bplaa-Phao', 'Som-Dtam', 'Khai-Loke-Khoei', 'Khaa-Mhoo', 'Su-Ki', 'Dtom-Kha-Gai', 'Dtom-Yam-Goong', 'Yam-Woon-Sen', 'Yen-Dta-Fo']
eng_trans = ['Stuffed Bitter Melon Soup', 'Stir-fried Crab with Curry Powder', 'Thai Curried Fish Cakes', 'Thai Dumplings', 'Thai Five-Spice Egg and Pork Stew', 'Deep Fried Chicken', 'Stir-fried Kale (Chinese Broccoli)', 'Crispy Fried Mussel Pancake', 'Mixed Clear Soup', 'Green Curry', 'Grilled Chicken', 'Baked Prawns with Glass Noodles', 'Grilled Prawns', 'Grilled Squid', 'Steamed cockles', 'Stir-fried Clams with Chili Paste', 'Thai Rice Porridge', 'Thai Deep Fried Omelette with Minced Pork', 'Thai Steamed Egg', 'Hainanese Chicken Rice', 'Thai Sweet Red Pork with Rice', 'Rice Noodles in Coconut Fish Curry', 'Herbal Thai Biryani with Chicken', 'Fried Pork with Garlic and Rice', 'Mango Sticky Rice', 'Fried Rice in Shrimp Paste', 'Grilled Pork Neck', 'Southern Thai Dry Curry', 'Tradtional Thick Noodle Soup', 'Boat Noodle (Blood Soup Noodle)', 'Spicy Minced Pork Salad', 'Massaman Chicken Curry', 'Pork Satay', 'Grilled Pork Spicy Salad', 'Stir-fried Morning Glory', 'Stir-fried Mixed Vegetables', 'Pad Thai', 'Stir-fried Bitter Melon Sprouts', 'Stir-fried Thai Basil with Meat', 'Noodles in Gravy with Pork', 'Roasted Duck', 'Roasted Fish', 'Thai Papaya Salad', 'Sweet Stewed Egg (Son-In-Law Egg)', 'Pork Leg Stew', 'Thai Hot Pot', 'Chicken in Coconut Soup', 'Thai Herbal Spicy Soup', 'Glass Noodle Salad', 'Pink Noodle Soup (Tofu Soup)']
ingredients = {
    "ต้มฟัก": ["winter melon", "pork ribs", "garlic", "salt", "pepper", "cilantro"],
    "ปูผัดผงกะหรี่": ["crab", "curry powder", "egg", "onion", "celery", "green onions", "garlic", "fish sauce", "soy sauce"],
    "ทอดมันปลา": ["fish paste", "red curry paste", "egg", "kaffir lime leaves", "green beans", "sugar", "fish sauce"],
    "ขนมจีบ": ["minced pork", "shrimp", "green onions", "wonton wrappers", "fried garlic", "soy sauce", "sesame oil"],
    "ไข่พะโล้": ["egg", "pork belly", "five-spice powder", "dark soy sauce", "sugar", "garlic", "cinnamon"],
    "ไก่ทอด": ["chicken", "flour", "garlic", "pepper", "salt", "oil"],
    "ผัดคะน้า": ["Chinese broccoli", "garlic", "soy sauce", "oyster sauce", "sugar"],
    "หอยทอด": ["oyster", "flour", "egg", "bean sprouts", "garlic chives", "fish sauce", "chili sauce"],
    "แกงจืด": ["tofu", "minced pork", "celery", "cabbage", "carrot", "glass noodles", "salt", "pepper"],
    "แกงเขียวหวาน": ["chicken", "green curry paste", "coconut milk", "eggplant", "sweet basil", "kaffir lime leaves"],
    "ไก่ย่าง": ["chicken", "lemongrass", "garlic", "coriander root", "soy sauce", "fish sauce", "pepper"],
    "กุ้งอบวุ้นเส้น": ["shrimp", "glass noodles", "ginger", "garlic", "black pepper", "celery", "soy sauce"],
    "กุ้งเผา": ["shrimp", "salt", "lime", "seafood sauce"],
    "ปลาหมึกย่าง": ["squid", "salt", "lime", "seafood sauce"],
    "หอยแครงลวก": ["cockles", "salt", "seafood sauce"],
    "หอยลายผัดพริกเผา": ["clams", "chili paste", "basil", "garlic", "sugar", "fish sauce"],
    "โจ๊ก": ["rice", "minced pork", "ginger", "green onions", "egg", "soy sauce"],
    "ไข่เจียวหมูสับ": ["egg", "minced pork", "fish sauce", "pepper", "oil"],
    "ไข่ตุ๋น": ["egg", "fish sauce", "green onions", "pepper"],
    "ข้าวมันไก่": ["chicken", "rice", "ginger", "cucumber", "cilantro", "soy sauce"],
    "ข้าวหมูแดง": ["pork", "rice", "red sauce", "boiled egg", "cucumber", "scallions"],
    "ขนมจีนน้ำยากระทิ": ["rice noodles", "coconut milk", "fish", "red curry paste", "lemongrass", "kaffir lime leaves"],
    "ข้าวหมกไก่": ["chicken", "rice", "yogurt", "spices", "cucumber", "mint", "fried shallots"],
    "ข้าวหมูทอดกระเทียม": ["pork", "garlic", "soy sauce", "rice", "cucumber"],
    "ข้าวเหนียวมะม่วง": ["sticky rice", "mango", "coconut milk", "sugar", "salt"],
    "ข้าวคลุกกะปิ": ["rice", "shrimp paste", "pork", "green mango", "shallots", "chili", "fried egg"],
    "คอหมูย่าง": ["pork neck", "garlic", "soy sauce", "pepper"],
    "คั่วกลิ้ง": ["minced pork", "kaffir lime leaves", "lemongrass", "chili paste", "basil"],
    "ก๋วยจั๊บ": ["rice noodles", "pork belly", "tofu", "coriander", "pepper", "egg"],
    "ก๋วยเตี๋ยวเรือ": ["rice noodles", "beef", "blood", "garlic", "bean sprouts", "green onions", "cilantro"],
    "ลาบหมู": ["minced pork", "mint", "chili flakes", "lime", "fish sauce", "onion"],
    "มัสมั่นไก่": ["chicken", "potatoes", "coconut milk", "peanuts", "masaman curry paste"],
    "หมูสะเต๊ะ": ["pork", "coconut milk", "turmeric", "curry powder", "peanut sauce"],
    "น้ำตกหมู": ["pork", "mint", "onion", "lime", "chili powder", "fish sauce"],
    "ผัดผักบุ้ง": ["morning glory", "garlic", "chili", "soy sauce", "oyster sauce"],
    "ผัดผักรวมมิตร": ["mixed vegetables", "garlic", "oyster sauce", "soy sauce"],
    "ผัดไท": ["rice noodles", "shrimp", "tofu", "bean sprouts", "peanuts", "tamarind sauce"],
    "ผัดยอดมะระ": ["bitter melon shoots", "garlic", "oyster sauce", "soy sauce"],
    "ผัดกะเพรา": ["minced pork", "holy basil", "chili", "garlic", "soy sauce", "fish sauce"],
    "ราดหน้าหมู": ["pork", "rice noodles", "broccoli", "carrot", "garlic", "soy sauce"],
    "เป็ดย่าง": ["duck", "honey", "soy sauce", "five-spice powder", "garlic"],
    "ปลาเผา": ["fish", "lemongrass", "salt", "lime", "seafood sauce"],
    "ส้มตำ": ["papaya", "tomato", "peanuts", "lime", "chili", "fish sauce"],
    "ไข่ลูกเขย": ["boiled egg", "tamarind sauce", "fried shallots", "chili"],
    "ขาหมู": ["pork leg", "five-spice powder", "soy sauce", "sugar", "garlic", "cinnamon"],
    "สุกี้": ["glass noodles", "pork", "vegetables", "tofu", "suki sauce"],
    "ต้มข่าไก่": ["chicken", "galangal", "coconut milk", "lemongrass", "mushrooms", "lime leaves"],
    "ต้มยำกุ้ง": ["shrimp", "mushrooms", "lemongrass", "galangal", "lime leaves", "chili paste"],
    "ยำวุ้นเส้น": ["glass noodles", "ground pork", "shrimp", "celery", "onion", "chili"],
    "เย็นตาโฟ": ["rice noodles", "fish balls", "fried tofu", "morning glory", "red sauce"]
}
ingredients_df = pd.DataFrame(list(ingredients.items()), columns=['th_names', 'ingredients'])
spicy_lv = [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 3, 0, 1, 3, 1, 0, 3, 1, 0, 0, 1, 3, 0, 0, 0, 3, 0, 0, 1, 1, 3, 2, 0]

mapped_df['class_names'] = class_names
mapped_df['th_names'] = th_names
mapped_df['th_pronounce'] = th_pronounce
mapped_df['eng_trans'] = eng_trans
mapped_df['spicy_lv'] = spicy_lv

mapped_df = mapped_df.merge(
    ingredients_df,
    on = 'th_names')

chilli_lv = {
    0 : ' not spicy',
    1 : ' :hot_pepper:',
    2 : ' :hot_pepper::hot_pepper:',
    3 : ' :fire::hot_pepper::hot_pepper::hot_pepper::fire:'
}

face_lv = {
    0 : ' :drooling_face:',
    1 : ' :face_savoring_food:',
    2 : ' :anxious_face_with_sweat:',
    3 : ' :hot_face:'
}


# Load the banner image
banner_image = Image.open("/content/drive/MyDrive/cnn-thfood/banner1.png")  # Replace with your image path

# Display the banner at the top of the app
st.image(banner_image, use_column_width=True)

# Center the title and description using HTML and CSS
st.markdown("""
    <style>
    .centered-text {
        text-align: center;
        font-size: 28px;
        font-weight: bold;
        margin-top: 20px;
    }
    .centered-description {
        text-align: center;
        font-size: 18px;
        color: white;
    }
     .centered-detected-as {
        text-align: center;
        font-size: 18px;
        font-weight: bold;
        color: white;
        margin-top: 20px;
    }
    .centered-classified-name {
        text-align: center;
        font-size: 24px;
        font-weight: bold;
        color: white;
        margin-top: 20px;
    }
     .centered-spicy-as {
        text-align: center;
        font-size: 18px;
        font-weight: bold;
        color: red;
        margin-top: 20px;
    }
    .centered-spicy {
        text-align: center;
        font-size: 30px;
        font-weight: bold;
        color: red;
        margin-top: 20px;
    }
    </style>
    """, unsafe_allow_html=True)

# Display title and description text in the center
st.markdown('<p class="centered-text">DISH DETECTIVE</p>', unsafe_allow_html=True)
st.markdown('<p class="centered-description">Classify Thai food images and discover details about each dish!</p>', unsafe_allow_html=True)

# Function to preprocess input image
def preprocess_image(uploaded_file):
  # Open the uploaded file as an image
  img = Image.open(uploaded_file)

  img = img.resize((224, 224))
  img_array = img_to_array(img) # Convert the image to a NumPy array
  img_array = tf.keras.applications.mobilenet_v3.preprocess_input(img_array)
  processed_image = np.expand_dims(img_array, axis=0)

  return processed_image

# Upload options: file and camera
input_image = st.file_uploader("Upload an image of Thai Food...", type=["jpg", "jpeg", "png"])

if input_image is not None:

    uploaded_image = Image.open(input_image)
    # Create three columns
    col1, col2, col3 = st.columns([1, 1, 1])
    with col2:  # Center column
      st.image(uploaded_image, caption="Uploaded Image", width = 250)

    img = preprocess_image(input_image)

    prediction = model.predict(img)
    pred_class = np.argmax(prediction, axis=1)[0]

    th_names = mapped_df['th_names'][pred_class]
    th_pronounce = mapped_df['th_pronounce'][pred_class]
    eng_trans = mapped_df['eng_trans'][pred_class]
    spicy_lv = mapped_df['spicy_lv'][pred_class]

    # Display the classified name in the center
    st.markdown(f'<p class="centered-detected-as">Dish detected as</p>', unsafe_allow_html=True)

    st.markdown(f'''
    <p class="centered-classified-name">
        {th_names} <br>
        {th_pronounce} <br>
        <span style="font-size:18px; color: white;"> Translation: - {eng_trans} -</span>
    </p>
    ''', unsafe_allow_html=True)

    tts = gTTS(text=th_names, lang='th')
    col11, col12, col13 = st.columns([1, 1, 1])
    with col12:
      # Use a temporary file to save and play the audio
      with tempfile.NamedTemporaryFile(delete=True, suffix=".mp3") as temp_file:
        tts.save(temp_file.name)
        # Play the generated audio directly in Streamlit
        st.audio(temp_file.name, format="audio/mp3", start_time=0)


    face_emoj = emoji.emojize(face_lv[spicy_lv])
    chilli_emoj = emoji.emojize(chilli_lv[spicy_lv])
    st.markdown(f'<p class="centered-spicy-as">Authentic Spicy Level</p>', unsafe_allow_html=True)
    st.markdown(f'<p class="centered-spicy">{chilli_emoj} {face_emoj}</p>', unsafe_allow_html=True)

    # Display ingredients as a bulleted list
    st.markdown(f'<p class="centered-detected-as">Expected Ingredients</p>', unsafe_allow_html=True)
    ing_list = mapped_df['ingredients'][pred_class]
    col_1, col_2, col_3 = st.columns([1, 1, 1])
    with col_2:
      for i in ing_list:
        st.write(f"• {i}")


Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txyt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.247.66.125
your url is: https://chatty-pigs-hug.loca.lt


### other ways to run Streamlit

In [ ]:
#!streamlit run app.py --server.port 8501

In [ ]:
# Run Streamlit in the background
#!streamlit run app.py &>/content/logs.txt &

# Set up LocalTunnel to expose the app
#!npx localtunnel --port 8501 > url.txt 2>&1 &